In [1]:
! ls -alh /var/genie-in-the-box/src/conf/long-term-memory/

total 40K
drwxr-xr-x 2 1000 1000 4.0K Aug  9 19:33 .
drwxr-xr-x 4 1000 1000 4.0K Jul 19 20:04 ..
-rw-r--r-- 1 1000 1000 4.0K Jul 19 20:04 ._events.jsonl
-rw-r--r-- 1 1000 1000  16K Aug  9 19:33 events.csv
-rw-r--r-- 1 1000 1000 9.9K Aug  9 19:33 events.jsonl


In [2]:
# ! pip install python-dotenv
! pip install langchain
! pip install langchain[docarray]
! pip install tiktoken
! pip install pinecone-client
! pip install jq
! pip install chromadb
! pip install openai[embeddings]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 3.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.7/239.7 kB 38.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 13.5 MB/s eta 0:00:00
  Created wheel for hnswlib: filename=hnswlib-0.7.0-cp310-cp310-linux_x86_64.w

In [3]:
import os

In [3]:
# Import prompt and define PromptTemplate

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI( model_name="gpt-4-0613", temperature=0.0 )


In [4]:
path = "/var/genie-in-the-box/src/lib"
sys.path.append( path )
import util as du
du.add_to_path( path )
# sys.path

Path [/var/genie-in-the-box/src/lib] already in sys.path


In [5]:
# import util as du
du.print_banner( "foo?" )

------------------------------------------------------------------------------------------------------------------------
- foo?
------------------------------------------------------------------------------------------------------------------------



In [56]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory( memory_key="chat_history", return_messages=True )

In [57]:
qa = ConversationalRetrievalChain.from_llm(
    llm,
    in_memory_jsonl_index.vectorstore.as_retriever(),
    memory=memory
)

In [6]:
import pandas as pd

In [7]:
# Load CSV data From file into the Pandas DataFrame.
df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   start_date           200 non-null    object
 1   end_date             200 non-null    object
 2   duration             200 non-null    int64 
 3   start_time           200 non-null    object
 4   end_time             200 non-null    object
 5   event                200 non-null    object
 6   recurrent            200 non-null    bool  
 7   recurrence_interval  50 non-null     object
 8   description          200 non-null    object
 9   priority_level       200 non-null    object
dtypes: bool(1), int64(1), object(8)
memory usage: 14.4+ KB


### Count events by type, manually. Brute force!

In [9]:
def get_event_count( df, event ):

    print( "Counting [{}] events...".format( event ), end="\n\n" )

    count = df[ df.event == event ].shape[ 0 ]

    print()
    print( "Done! Total {} {} events".format( count, event ) )

get_event_count( df, "Birthday" )

Counting [Birthday] events...


Done! Total 17 Birthday events


In [10]:
from langchain.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent

llm = ChatOpenAI( model_name="gpt-4-0613", temperature=0.0 )

raw_df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )


In [30]:
template_string = """
You are an event planner with proven capacity for querying pandas dataframes.
I expect you to query the dataframe provided and answer in plain English unless specifically requested to do otherwise.
NEVER display raw dataframe content, unless specifically requested to do otherwise.
All dates and temporal references like 'today', 'yesterday', 'next week' should all be calculated by using Python's datetime library.
When I ask you about events contained in this dataframe, you will always sort the events in ascending order, 1 per line, unless specifically requested to do otherwise.
If the question, delimited by three hash symbols ###, is vague or difficult to answer, please ask for clarification.
Given that, answer the following question: ###{question}###
"""

event_prompt_template = ChatPromptTemplate.from_template( template_string )
# print( prompt_template, end="\n\n" )

event_prompt = event_prompt_template.format_messages( question="How many records are in this dataset?" )
print( event_prompt, end="\n\n" )

# response_autoencoder = llm( prompt_autoencoder )
# print( response_autoencoder.content, end="\n\n" )

[HumanMessage(content="\nYou are an event planner with proven capacity for querying pandas dataframes.\nI expect you to query the dataframe provided and answer in plain English unless specifically requested to do otherwise.\nNEVER display raw dataframe content, unless specifically requested to do otherwise.\nAll dates and temporal references like 'today', 'yesterday', 'next week' should all be calculated by using Python's datetime library.\nWhen I ask you about events contained in this dataframe, you will always sort the events in ascending order, 1 per line, unless specifically requested to do otherwise.\nIf the question, delimited by three hash symbols ###, is vague or difficult to answer, please ask for clarification.\nGiven that, answer the following question: ###How many records are in this dataset?###\n", additional_kwargs={}, example=False)]



In [31]:
df_agent = create_pandas_dataframe_agent( llm, raw_df, verbose=True )

In [14]:
event_prompt = event_prompt_template.format_messages( question="How many records are in this dataset?" )
df_agent.run( event_prompt )



> Entering new AgentExecutor chain...
Thought: To find out how many records are in the dataset, I can use the `shape` attribute of the dataframe. The `shape` attribute returns a tuple representing the dimensionality of the DataFrame. The first element of the tuple will give us the number of rows, which is the number of records in the dataset.
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 200
Thought:I now know the final answer
Final Answer: There are 200 records in this dataset.

> Finished chain.


'There are 200 records in this dataset.'

In [32]:
# Create function to take a question and call the data frame agent.
def query_dataframe( question ):

    event_prompt = event_prompt_template.format_messages( question=question )
    response = df_agent.run( event_prompt )

    return response

response = query_dataframe( "What day is today? What day was yesterday? What day is tomorrow?" )
response



> Entering new AgentExecutor chain...
Thought: The question is asking for today's date, yesterday's date, and tomorrow's date. I can use Python's datetime library to get these dates.
Action: python_repl_ast
Action Input: 
```python
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()

# Get yesterday's date
yesterday = today - timedelta(days=1)

# Get tomorrow's date
tomorrow = today + timedelta(days=1)

today, yesterday, tomorrow
```
Observation: (datetime.datetime(2023, 8, 10, 18, 43, 8, 736252), datetime.datetime(2023, 8, 9, 18, 43, 8, 736252), datetime.datetime(2023, 8, 11, 18, 43, 8, 736252))
Thought:I now know the dates for today, yesterday, and tomorrow.
Final Answer: Today is August 10, 2023. Yesterday was August 9, 2023. Tomorrow will be August 11, 2023.

> Finished chain.


'Today is August 10, 2023. Yesterday was August 9, 2023. Tomorrow will be August 11, 2023.'

In [16]:
response = query_dataframe( "How many different types of events do I have?" )
response



> Entering new AgentExecutor chain...
Thought: To find out how many different types of events there are, I need to find the unique values in the 'event' column of the dataframe.
Action: python_repl_ast
Action Input: df['event'].nunique()
Observation: 13
Thought:I now know the final answer
Final Answer: There are 13 different types of events.

> Finished chain.


'There are 13 different types of events.'

In [17]:
response = query_dataframe( "How many TODO events do I have?" )
response



> Entering new AgentExecutor chain...
Thought: To find out how many TODO events there are, I need to filter the dataframe where the 'event' column is equal to 'TODO' and then count the number of rows.
Action: python_repl_ast
Action Input: df[df['event'] == 'TODO'].shape[0]
Observation: 21
Thought:I now know the final answer
Final Answer: You have 21 TODO events.

> Finished chain.


'You have 21 TODO events.'

In [18]:
get_event_count( df, "TODO" )

Counting [TODO] events...


Done! Total 21 TODO events


In [20]:
response = query_dataframe( "Count and display my birthday events in August" )
response



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to filter the dataframe to only include events that occur in August and have 'birthday' in their description. Then, I can count the number of these events.
Action: python_repl_ast
Action Input: df[(df['start_date'].str.contains('-08-')) & (df['description'].str.contains('birthday'))].shape[0]
Observation: AttributeError: Can only use .str accessor with string values!
Thought:The error suggests that the 'start_date' column is not of string type. I need to convert it to string before applying the str.contains() function.
Action: python_repl_ast
Action Input: df['start_date'] = df['start_date'].astype(str)
df[(df['start_date'].str.contains('-08-')) & (df['description'].str.contains('birthday'))].shape[0]
Observation: 11
Thought:I now know the final answer
Final Answer: There are 11 birthday events in August.

> Finished chain.


'There are 11 birthday events in August.'

In [35]:
response = query_dataframe( "Show me birthday events in August" )
response



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to filter the dataframe to only include rows where the 'event' column is 'Birthday' and the 'start_date' is in August. I can use the pandas `str.contains` method to filter for 'Birthday' events and the `dt.month` attribute to filter for August dates.
Action: python_repl_ast
Action Input: df[(df['event'].str.contains('Birthday')) & (pd.to_datetime(df['start_date']).dt.month == 8)].sort_values('start_date')
Observation: NameError: name 'pd' is not defined
Thought:I forgot to import the pandas library. I need to import it before I can use it.
Action: python_repl_ast
Action Input: import pandas as pd
Observation: 
Thought:Now that I have imported the pandas library, I can try to filter the dataframe again.
Action: python_repl_ast
Action Input: df[(df['event'].str.contains('Birthday')) & (pd.to_datetime(df['start_date']).dt.month == 8)].sort_values('start_date')
Observation:      start_date    end_date  duratio

"Here are the birthday events in August:\n\n1. Gregorio (friend)'s birthday on 2023-08-02\n2. Sally (sister)'s birthday on 2023-08-02\n3. John (coworker)'s birthday on 2023-08-07\n4. Pablo (friend)'s birthday from 2023-08-10 to 2023-08-11\n5. Jenny (coworker)'s birthday on 2023-08-12\n6. Gregorio (friend)'s birthday on 2023-08-13\n7. Bob (brother)'s birthday from 2023-08-14 to 2023-08-21\n8. Pablo (friend)'s birthday on 2023-08-16\n9. Bob (brother)'s birthday from 2023-08-22 to 2023-08-29\n10. Barbara Jane Ruiz (mother)'s birthday on 2023-08-24\n11. Inash (girlfriend)'s birthday from 2023-08-27 to 2023-08-30"

In [43]:
events = response.split( "\n\n" )[ 1: ]
for event in events: print( event )

1. Gregorio (friend)'s birthday on 2023-08-02
2. Sally (sister)'s birthday on 2023-08-02
3. John (coworker)'s birthday on 2023-08-07
4. Pablo (friend)'s birthday from 2023-08-10 to 2023-08-11
5. Jenny (coworker)'s birthday on 2023-08-12
6. Gregorio (friend)'s birthday on 2023-08-13
7. Bob (brother)'s birthday from 2023-08-14 to 2023-08-21
8. Pablo (friend)'s birthday on 2023-08-16
9. Bob (brother)'s birthday from 2023-08-22 to 2023-08-29
10. Barbara Jane Ruiz (mother)'s birthday on 2023-08-24
11. Inash (girlfriend)'s birthday from 2023-08-27 to 2023-08-30


In [34]:

response = query_dataframe( "What have I asked you about thus far?" )
response



> Entering new AgentExecutor chain...
Thought: The question is asking for a summary of what has been asked so far. Since this is the first question, nothing has been asked about the dataframe yet.
Final Answer: You haven't asked anything about the dataframe yet.

> Finished chain.


"You haven't asked anything about the dataframe yet."

In [111]:
df_agent.run( "Do you remember the August birthday events you just showed me? If so, only print their name and their date in plain english, in descending order, one per line" )



> Entering new AgentExecutor chain...
Thought: The question is asking for events that are birthdays in August. However, the dataframe does not have a column that directly indicates whether an event is a birthday or not. The closest thing we have is the 'description' column, which might contain the word 'birthday'. We also don't have any events in August in the provided data. However, assuming we have a larger dataset, we can filter the dataframe for events that contain the word 'birthday' in the 'description' column and occur in August. We can then print the 'description' and 'start_date' of these events in descending order. 

Action: python_repl_ast
Action Input: df[df['description'].str.contains('birthday') & (df['start_date'].str.slice(5, 7) == '08')][['description', 'start_date']].sort_values('start_date', ascending=False)
Observation: AttributeError: Can only use .str accessor with string values!
Thought:The error suggests that not all values in the 'description' column are stri

"Here are the August birthday events in descending order:\n\nInash (girlfriend)'s birthday on August 27, 2023\nBarbara Jane Ruiz (mother)'s birthday on August 24, 2023\nBob (brother)'s birthday on August 22, 2023\nPablo (friend)'s birthday on August 16, 2023\nBob (brother)'s birthday on August 14, 2023\nGregorio (friend)'s birthday on August 13, 2023\nJenny (coworker)'s birthday on August 12, 2023\nPablo (friend)'s birthday on August 10, 2023\nJohn (coworker)'s birthday on August 7, 2023\nGregorio (friend)'s birthday on August 2, 2023\nSally (sister)'s birthday on August 2, 2023"

In [114]:
foo = "Here are the August birthday events in descending order:\n\nInash (girlfriend)'s birthday on August 27, 2023\nBarbara Jane Ruiz (mother)'s birthday on August 24, 2023\nBob (brother)'s birthday on August 22, 2023\nPablo (friend)'s birthday on August 16, 2023\nBob (brother)'s birthday on August 14, 2023\nGregorio (friend)'s birthday on August 13, 2023\nJenny (coworker)'s birthday on August 12, 2023\nPablo (friend)'s birthday on August 10, 2023\nJohn (coworker)'s birthday on August 7, 2023\nGregorio (friend)'s birthday on August 2, 2023\nSally (sister)'s birthday on August 2, 2023"

foo = foo.split( "\n" )
foo.reverse()

for f in foo:
    print( f )

Sally (sister)'s birthday on August 2, 2023
Gregorio (friend)'s birthday on August 2, 2023
John (coworker)'s birthday on August 7, 2023
Pablo (friend)'s birthday on August 10, 2023
Jenny (coworker)'s birthday on August 12, 2023
Gregorio (friend)'s birthday on August 13, 2023
Bob (brother)'s birthday on August 14, 2023
Pablo (friend)'s birthday on August 16, 2023
Bob (brother)'s birthday on August 22, 2023
Barbara Jane Ruiz (mother)'s birthday on August 24, 2023
Inash (girlfriend)'s birthday on August 27, 2023

Here are the August birthday events in descending order:


In [104]:
from langchain.agents.agent_toolkits.csv.base import create_csv_agent

csv_agent = create_csv_agent( llm=llm, path=du.get_project_root() + "/src/conf/long-term-memory/events.csv", verbose=True )

In [105]:
csv_agent.run( "How many records are in this dataset?" )



> Entering new AgentExecutor chain...
Thought: To find out the number of records in the dataset, I can use the `shape` attribute of the dataframe. The first element of the shape attribute will give me the number of rows, which is the number of records in the dataset.
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 200
Thought:I now know the final answer.
Final Answer: There are 200 records in this dataset.

> Finished chain.


'There are 200 records in this dataset.'

In [106]:
csv_agent.run( "How many birthday events are in this dataset?" )



> Entering new AgentExecutor chain...
Thought: To find out how many birthday events are in the dataset, I need to filter the dataframe by the 'event_type' column and count the number of rows where 'event_type' is 'Birthday'.
Action: python_repl_ast
Action Input: df[df['event_type'] == 'Birthday'].shape[0]
Observation: 17
Thought:I now know the final answer.
Final Answer: There are 17 birthday events in this dataset.

> Finished chain.


'There are 17 birthday events in this dataset.'

In [107]:
csv_agent.run( "Show me all birthdays in August" )



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `Thought: The dataframe does not contain any column that directly indicates whether an event is a birthday or not. Therefore, it is not possible to filter out birthdays in August from this dataframe.`

In [143]:
csv_agent.run( "Today is July 29th, 2023. Show me what events I have for today. Show me the date, start time, event type, and description for all events. Format using Markdown. Use tool python_repl_ast." )



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include events that are happening today. Then I can select the columns I need and format the output using Markdown.

Action: python_repl_ast
Action Input: df['date'] == '2023-07-29'
Observation: 0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9      True
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18     True
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58    False
59    Fal

"The events for today are:\n\n|    | date       | start_time   | event_type   | description                |\n|---:|:-----------|:-------------|:-------------|:---------------------------|\n|  9 | 2023-07-29 | 00:00        | Birthday     | Leroy Ruiz (father)'s birthday |\n| 18 | 2023-07-29 | 00:00        | Subscription | This is a subscription.    |"